In [ ]:
from data_gatherer.data_gatherer import DataGatherer
import pandas as pd
from data_gatherer.llm.response_schema import *

In [ ]:
dg = DataGatherer(log_level="INFO", load_from_cache=False, save_to_cache=True)

In [ ]:
df = pd.read_csv("scripts/NYC Open Data/NYC Open Data - Benchmark - paper-dataset.csv")
prompt_name = "GPT_F2DR_FewShot_Descr"
subdir='metadata_prompts'
response_schema = autoDDG_from_context_schema
grobid_for_pdf=True
FDR=True

In [ ]:
for index, row in df.iterrows():
    paper_link = row['Paper']
    if paper_link not in todo:
        continue
    title = row['Title']
    link = row['Link']
    dataset_details = f"Title: {title}\nLink: {link}"
    dg.logger.info(f"Target Dataset Details: {dataset_details}")
    full_text = dg.fetch_data(paper_link)
    dg.init_parser_by_input_type(dg.data_fetcher.raw_data_format, raw_data = full_text, grobid_for_pdf = grobid_for_pdf, full_document_read=FDR)
    src_cont = dg.normalize_fulltext_input(full_text)
    article_id = dg.data_fetcher.url_to_article_id(paper_link)
    dg.logger.info(f"Processing article ID: {article_id}")
    dataset_info = dg.parser.extract_dataset_description(src_cont, dataset_details, subdir=subdir, prompt_name=prompt_name, response_schema=response_schema, article_id=article_id)[0]
    # add to each row the extracted description
    df.at[index, 'Extracted Description'] = dataset_info.get('dataset_description', '')
    df.at[index, 'reuse_potential'] = dataset_info.get('reuse_potential', '')
    df.at[index, 'citation_type'] = dataset_info.get('citation_type', '')

In [ ]:
df

In [ ]:
# add sub to transform any format to plain text
df.to_csv("scripts/NYC Open Data/NYC Open Data - Benchmark - paper-dataset-with-descriptions-1.csv", index=False)

In [ ]:
df['Paper'].to_list()